## Criando o primeiros imports do simulador

In [ ]:
from quantumnet.components import Network, Controller, Host
from quantumnet.objects import Logger, Qubit

from random import randint, choice

## Criando a rede

In [ ]:
def initNetwork(rows: int, columns: int, log: bool=True) -> tuple[Network, Controller]:
    """
    Will initiate the network and will define the controller

    Args:
        log: If True will activate logs od simulation  

    Returns:
        Will return the Network and the Controller
    """
    # Defining the network
    network = Network()

    # Defining the Controller
    controller = Controller(network)

    # Defining the topology
    network.set_ready_topology('Grade', rows, columns)

    # Draw Nodes
    network.draw()

    # Log of simulator
    if log:
        Logger.activate(Logger)
        
    return network, controller


## Escolhendo os nós de Alice, de Bob e do Atacante

In [ ]:
def selectNodes(network: Network) -> tuple[Host, Host, Host]:
    """
    Will define the nodes: Alice, Bob and Black_Hole

    Args:
        network: network to wich the nodes belong

    Returns:
        Hosts: Will return respectively: Alice host, Bob host, Black_Hole host
    """
    alice_id = randint(0, len(network.hosts)-1)
    alice = network.get_host(alice_id)
    print(f"O id de Alice é: {alice}")

    valid = False
    while not valid:
        bob_id = randint(0, len(network.hosts)-1)
        if bob_id != alice_id:
            valid = True

    bob = network.get_host(bob_id)
    print(f"O id de Bob é: {bob}")

    valid = False
    while not valid:
        black_hole_id = randint(0, len(network.hosts)-1)
        if black_hole_id != alice_id and black_hole_id != bob_id:
            valid = True

    black_hole = network.get_host(black_hole_id)
    print(f"O id do atacante é: {black_hole}")
    
    return alice, bob, black_hole


## Passando as rotas para o controlador

In [ ]:
def getRoute(network: Network, controller: Controller, alice: Host, alice_id: int, bob: Host, bob_id: int) -> list:
    """
    Will define the route of Alice to Bob

    Args:
        network: Network to wich the nodes belong
        controller: Controller of Network
        alice: Sender host
        alice_id: Sender host number
        bob: Receiver host
        bob_id: Receiver host number

    Returns:
        Dict: Dict with the rotes of Alice to Bob
    """
    # Geting the all routing table
    controller.register_routing_tables()

    # Defining the route of Alice to Bob
    route = network.get_host(alice_id).routing_table[bob_id]
    print(f"Alice: {alice} deseja se comunicar com Bob: {bob} pela rota {route}")
    
    return route

## Modificando manualmente a probabilidade do Black_Hole

In [ ]:
def setNetworkSwappProb(network: Network, number_nodes: int, network_prob: float,  malicious_host: Host, malicious_host_prob: float) -> None:
    for host_id in range(0, number_nodes):
        temp_host = network.get_host(host_id=host_id)
        if temp_host == malicious_host:
            temp_host.setEntanglementSwappingProb(malicious_host_prob)
        else:
            temp_host.setEntanglementSwappingProb(network_prob)


## Criação manual de entanglement entre os nós

In [ ]:
def addQubits(host_A: Host, host_B: Host, counter: int) -> int:
    temp_qubit_counter = counter
    qubit = Qubit(temp_qubit_counter)
    host_A.add_qubit(qubit)
    qubit = Qubit(temp_qubit_counter+1)
    host_B.add_qubit(qubit)
    temp_qubit_counter += 2

    return temp_qubit_counter
    
# Creating entanglement between neighbors hosts
def createEntanglements(route: list, network: Network, number_of_entanglements: int) -> None:
    """
    Create entangled pair between the hosts of route

    Args:
        route: Dict with the route of Alice to Bob
        network: Network to wich the nodes belong
        number_of_entanglements: Number of desired pairs
    """
    # Used qubits
    qubit_counter = 0

    # Loop to create new entanglements
    for entanglement in range(0, number_of_entanglements):
        
        # Save the qubit index to create new qubits
        temp_qubit_counter = qubit_counter

        # Create new entanglement to every host in the route
        for host_number in range(0, len(route)):
            if host_number != len(route) - 1:
                host_A = network.get_host(route[host_number])
                host_B = network.get_host(route[host_number+1])

                # Will trying until entangled be successfully created
                entangled = False
                while not entangled:

                    # If dont't have qubit on memory will add
                    if host_A.memory == [] or host_B.memory == []:
                        temp_qubit_counter = addQubits(host_A=host_A, host_B=host_B, counter=temp_qubit_counter)
                    print(f"Tentativa de entanglement entre {host_A} e {host_B}")

                    # Trying do entanglement between host_A and host_B
                    entangled = network.physical.entanglement_creation_heralding_protocol(host_A, host_B)

                    if not entangled:
                        temp_qubit_counter = addQubits(host_A=host_A, host_B=host_B, counter=temp_qubit_counter)
                        
        # Update counter to qubit index
        qubit_counter += temp_qubit_counter

    print(f"Foram criados {qubit_counter} qubits a mais para a realização dos {entanglement+1} entanglements")

## Tentativa de Entanglement Swapping entre Alice e Bob

In [ ]:
def EntanglementSwapping(network: Network, alice_id: int, bob_id: int) -> bool:
    """
    Will try do Entanglement Swapping between Alice and Bob as long as there are pairs

    Args:
        network: Network to wich the nodes belong
        alice_id: Number of Alice host
        bob_id: Number of Bob host
    """
    entangled = 0
    while entangled == 0:
        # Trying entanglement swapping between Alice and Bob
        entangled = network.networklayer.entanglement_swapping(alice_id, bob_id)

    # Signaling that the application is finished to controller
    network.get_host(alice_id).announce_to_controller_app_has_finished()

    if entangled == -1:
        print("É impossível realizar o entanglement")
        return False
    
    return True

## Criando a simulação

In [ ]:
def run(runTimes: int, log: bool, rows: int, columns: int, create_new_entanglements: int = 0, network_prob: float = None, black_hole_prob: int = None) -> dict:
    """
    Run the simulation with the desired parameters

    Args:
        runTimes: Number of times of simulation will run
        log: Will show the simulator logs
        create_new_entanglements: Number of entangled pair will create
        black_hole_fidelity: Fidelity desired to attack

    Returns:
        Dict: Return every pairs information of run on simulation
    """
    data = {}
    for run in range(0, runTimes):
        # Create network and controller
        network, controller = initNetwork(rows=rows, columns=columns, log=log)

        # Defining the nodes
        alice, bob, black_hole = selectNodes(network=network)
        alice_id = alice.host_id
        bob_id = bob.host_id

        # Defining route
        route = getRoute(network=network, controller=controller, alice=alice, alice_id=alice_id, bob=bob, bob_id=bob_id)
        controller.announce_to_route_nodes(route)
        controller.announce_to_alice_and_bob(route)

        print(f"Essa é a rota: {route}")

        possible_black_hole = route.copy()
        possible_black_hole.pop()
        if len(possible_black_hole) > 0:
            possible_black_hole.pop(0)
        print("Possível lista do black hole", possible_black_hole)
        if len(possible_black_hole) > 0:
            black_hole = network.get_host(choice(possible_black_hole)) 
            print(f"Esse é o Black Hole: {black_hole.host_id}")
            black_hole._prob_entanglement_swapping = black_hole_prob
        else:
            print("Sem a presença de um Black Hole")


        if black_hole_prob is not None:
            setNetworkSwappProb(network=network, number_nodes=rows*columns, network_prob=network_prob, malicious_host=black_hole, malicious_host_prob=black_hole_prob)


        # Will create entanglements between Alice and Bob
        if create_new_entanglements > 0:
            createEntanglements(route=route, network=network, number_of_entanglements=create_new_entanglements)

        # Do entanglement swapping between ALice and Bob
        entangled = EntanglementSwapping(network=network, alice_id=alice_id, bob_id=bob_id)

        # Collect data for the simulation run
        used_eprs = network.get_total_useds_eprs()
        total_eprs = network.get_eprs()
        number_of_eprs = {}
        for key in total_eprs:
            number_of_eprs[key] = len(total_eprs[key])
        data[f'run: {run}'] = {"Entrelaçado": entangled, "Alice & Bob": (alice_id, bob_id), 'Eprs usados': used_eprs, "Eprs criados": create_new_entanglements, 'Total Eprs': number_of_eprs}
    
    return data

## Rodando a simulação e coletando os dados

In [ ]:
data = run(runTimes=100, rows=4, columns=3, log=True, create_new_entanglements=0, black_hole_prob=0.9, network_prob=0.9)

## Mostrando os dados coletados

In [ ]:
data

## Taxa de Sucesso

In [ ]:
success = 0
for run in data:
    if data[run]['Entrelaçado']:
        success += 1
tax = success/len(data.keys()) * 100
print(f'{tax}%')